<a href="https://colab.research.google.com/github/shubhamscifi/Instacart-Market-Basket-Analysis/blob/main/5.%20Auto-Encoder%20for%20Ftr%20Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[Instacart Market Basket Analysis](https://www.kaggle.com/c/instacart-market-basket-analysis/)**

---
- Given order_id predict all the products that the user will reorder.

---








## Importing libraries

In [ ]:
import plotly
import plotly.express as px
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import re
import gc   # garbage collector
import pickle
# https://pypi.org/project/tqdm/#:~:text=jupyter%20console.%20Use-,auto,-instead%20of%20autonotebook
from tqdm.auto import tqdm
import time
from joblib import Parallel, delayed
from sklearn.metrics import f1_score,confusion_matrix,\
                            precision_recall_fscore_support,classification_report,\
                            accuracy_score,log_loss
from sklearn.model_selection import train_test_split

bold = lambda string: '\033[1m'+string+'\033[0m'    # for bold text
printb = lambda string: print('\033[1m'+string+'\033[0m')
# https://stackoverflow.com/questions/8924173/how-do-i-print-bold-text-in-python/8930747

## Kaggle file uploader utility:
- To upload intermediate tables.

In [ ]:
def kaggle_file_uploader(files,id='shubhamscifi/instacart',title='instacart',folder='data',msg='',first_time=False,del_after_upload=True):
    '''Uploads list of files to kaggle.
    Note: make sure to run after kaggle authentication.
    id : must be between 6-50 chars after "username/".
    title : must be between 6-50 chars.
    files : list of path of files that are to be uploaded.
    first_time: True if the data is being uploaded for the first time.
    del_after_upload: True if given folder needs to be deleted after file upload finishes.'''
    # https://github.com/Kaggle/kaggle-api

    # create data package json file
    !mkdir {folder}
    !kaggle datasets init -p {folder}

    # preparing metadata json file
    import json,os
    metadata = open(os.path.join(folder,'dataset-metadata.json'),'r+')
    meta = json.load(metadata)
    meta['id'] = id
    meta['title']= title
    metadata.seek(0)
    json.dump(meta,metadata)
    metadata.truncate()
    metadata.close()

    for file in set(files):
        !cp {file} {folder}

    # upload dataset to kaggle
    if (first_time):
        !kaggle datasets create -p {folder}
    else:
        # Create a New Dataset Version
        !kaggle datasets version -p {folder} -m '{msg}'

    if (del_after_upload):
        !rm -rf {folder}

## Loading Data

In [ ]:
# Kaggle authentication
from getpass import getpass
import os

os.environ['KAGGLE_USERNAME'] = "shubhamscifi" #input('Enter kaggle username: ') # kaggle username
os.environ['KAGGLE_KEY'] = getpass('Enter Token: ') # kaggle api key

Enter Token: ··········


**Download intermediate prepared tables.**

In [ ]:
!kaggle datasets download -d shubhamscifi/instacart --unzip

 99% 1.46G/1.47G [00:13<00:00, 189MB/s]
100% 1.47G/1.47G [00:13<00:00, 113MB/s]


In [ ]:
%%time
dataset = pd.read_feather('dataset.feather')

CPU times: user 2.92 s, sys: 19.8 s, total: 22.7 s
Wall time: 1.35 s


In [ ]:
# loading data into pandas dataframe
orders = pd.read_csv('/content/orders.csv',dtype={'order_id':np.uint32,
                                                  'user_id' :np.uint32,
                                                  'order_number':'uint8',
                                                  'order_hour_of_day':'uint8',
                                                  'order_dow':'uint8',
                                                  'days_since_prior_order':'float16'})
dep = pd.read_csv('/content/departments.csv', dtype={'department_id':'uint8',
                                                     'department': str})
aisles = pd.read_csv('/content/aisles.csv', dtype={'aisle_id':'uint8',
                                                     'aisle': str})
products = pd.read_csv('/content/products.csv', dtype={'aisle_id':'uint8',
                                                     'department_id':'uint8',
                                                     'product_name': str,
                                                     'product_id': np.uint16})
order_products_prior = pd.read_csv('/content/order_products__prior.csv',
                                   dtype={'add_to_cart_order':'uint8',
                                          'reordered':'uint8',
                                          'order_id':np.uint32,
                                          'product_id':np.uint16})
order_products_train = pd.read_csv('/content/order_products__train.csv',
                                   dtype={'add_to_cart_order':'uint8',
                                          'reordered':'uint8',
                                          'order_id':np.uint32,
                                          'product_id':np.uint16})

## Feature scaling:

In [ ]:
# list of features that are not in the range 0-1.
scaling_ftrs = ['#reorders_u', '#purchases_u', '#first_purchases_u',
       'mean_#reorders_u', 'median_#reorders_u', 'min_#reorders_u', 'max_#reorders_u',
       'mean_#purchases_u', 'median_#purchases_u', 'min_#purchases_u', 'max_#purchases_u', 
       'mean_#first_purchases_u', 'median_#first_purchases_u', 'min_#first_purchases_u', 
       'max_#first_purchases_u', '#avg_reorders_dep', '#avg_reorders_aisle', '#reorders_p',
       '#purchases_p', '#first_purchases_p', '#reorders_up']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# feature scaling
scaler = MinMaxScaler()
scaler.fit(dataset.loc[dataset.eval_set=='train',scaling_ftrs])

MinMaxScaler(copy=True, feature_range=(0, 1))

In [ ]:
dataset[scaling_ftrs] = (
    scaler.transform(dataset[scaling_ftrs])
)
dataset.head()

,user_id,product_id,order_id,eval_set,reordered,#reorders_u,#purchases_u,#first_purchases_u,p(reorder|user),mean_#reorders,median_#reorders,min_#reorders,max_#reorders,mean_#purchases,median_#purchases,min_#purchases,max_#purchases,mean_#first_purchases,median_#first_purchases,min_#first_purchases,max_#first_purchases,"mean_p(reorder|user,order)","median_p(reorder|user,order)","min_p(reorder|user,order)","max_p(reorder|user,order)",dep_target_enc,aisle_target_enc,eatable,#avg_reorders_dep,p(reorder|dep_of_prod),#avg_reorders_aisle,p(reorder|aisle_of_prod),#reorders_p,#purchases_p,#first_purchases_p,p(reorder|product),#reorders_up,"p(reorder|user,product)",reordered_in_last_order,reordered_in_2ndlast_order,reordered_in_3rdlast_order
0,2,13176,1492625,train,0.0,0.035002,0.062786,0.13931,0.476923,0.153297,0.191489,0.0,0.137255,0.183962,0.1625,0.072727,0.173611,0.144637,0.142857,0.028571,0.097561,0.482419,0.571429,0.0,0.888889,0.128464,0.169311,1,1.000000,0.649913,1.000000,0.718104,0.792539,0.802958,0.859117,0.832555,0.000000,0.000000,0,0,0
1,2,41787,1492625,train,1.0,0.035002,0.062786,0.13931,0.476923,0.153297,0.191489,0.0,0.137255,0.183962,0.1625,0.072727,0.173611,0.144637,0.142857,0.028571,0.097561,0.482419,0.571429,0.0,0.888889,0.128464,0.169311,1,1.000000,0.649913,1.000000,0.718104,0.057738,0.074936,0.167629,0.649903,0.010309,0.076923,0,0,0
2,2,32792,1492625,train,1.0,0.035002,0.062786,0.13931,0.476923,0.153297,0.191489,0.0,0.137255,0.183962,0.1625,0.072727,0.173611,0.144637,0.142857,0.028571,0.097561,0.482419,0.571429,0.0,0.888889,0.088779,0.098166,1,0.066800,0.574180,0.043716,0.591986,0.001984,0.002897,0.007816,0.577372,0.082474,0.615385,0,0,1
3,2,47766,1492625,train,0.0,0.035002,0.062786,0.13931,0.476923,0.153297,0.191489,0.0,0.137255,0.183962,0.1625,0.072727,0.173611,0.144637,0.142857,0.028571,0.097561,0.482419,0.571429,0.0,0.888889,0.128464,0.169311,1,1.000000,0.649913,1.000000,0.718104,0.336279,0.374159,0.578325,0.758103,0.030928,0.230769,0,0,0
4,2,20574,1492625,train,0.0,0.035002,0.062786,0.13931,0.476923,0.153297,0.191489,0.0,0.137255,0.183962,0.1625,0.072727,0.173611,0.144637,0.142857,0.028571,0.097561,0.482419,0.571429,0.0,0.888889,0.104139,0.104603,1,0.126903,0.607719,0.092866,0.606517,0.019121,0.021572,0.034778,0.747621,0.010309,0.076923,0,0,0




---



## Feature Extraction using **Denoising Auto-encoder**:
- To get non-linear relationships from the data.
- Denoising makes Auto-encoder more robust and prevent it from over-fitting.

In [ ]:
#importing tensorflow
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,InputLayer
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
tf.__version__

'2.5.0'

In [ ]:
import os
import numpy as np
import random as rn
os.environ['PYTHONHASHSEED'] = '0'

##https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
## Have to clear the session. If you are not clearing, Graph will create again and again and graph size will increses. 
## Varibles will also set to some value from before session
tf.keras.backend.clear_session()

## Set the random seed values to regenerate the model.
np.random.seed(0)
rn.seed(0)

In [ ]:
metadata = dataset.columns[:4]
# 4th index is the target variable
continuous_ftrs = list(dataset.columns[5:27]) + list(dataset.columns[28:38])
cat_ftrs = ['eatable',
            'reordered_in_last_order',
            'reordered_in_2ndlast_order',
            'reordered_in_3rdlast_order']
ftrs = continuous_ftrs + cat_ftrs

In [ ]:
print('No. of continous ftrs:'.ljust(24),len(continuous_ftrs))
print('No. of categorical ftrs:'.ljust(24),len(cat_ftrs))
print('Total no. of ftrs:'.ljust(24),len(ftrs))

No. of continous ftrs:   32
No. of categorical ftrs: 4
Total no. of ftrs:       36


In [ ]:
# tf_dataset_tr = tf.data.Dataset.from_tensor_slices(dataset.loc[dataset.eval_set=='train',ftrs])
# taking lot of RAM

In [ ]:
def noise_adder(x_train,y_train):
    '''Adds Gaussian noise to the input data.
    1. Makes auto-encoder robust to noise.
    2. Saves auto-encoder from overfitting.'''
    noise_factor = 0.2
    x_train_noisy = x_train + noise_factor * tf.random.normal(shape=x_train.shape,dtype=tf.float64) 
    x_train_noisy = tf.clip_by_value(x_train_noisy, clip_value_min=0., clip_value_max=1.,)
    return x_train_noisy,y_train

In [ ]:
def data_generator_tr():
    '''Construct a data generator for tf.data.Dataset'''
    data = dataset.loc[dataset.eval_set=='train',ftrs]
    for row in data.itertuples(index=False, name=None):
        yield row,row

In [ ]:
def data_generator_cv():
    '''Construct a data generator for tf.data.Dataset'''
    data = dataset.loc[dataset.eval_set=='cv',ftrs]
    for row in data.itertuples(index=False, name=None):
        yield row,row

In [ ]:
# setting different parameters
train_len = (dataset.eval_set=='train').sum()
cv_len = (dataset.eval_set=='cv').sum()
epochs = 5
batch_size = 32
buffer_size_tr = 10000
buffer_size_cv = 5000
steps_per_epoch = np.ceil(train_len/batch_size)
validation_steps = np.ceil(cv_len/batch_size)
repeat_count = np.ceil((steps_per_epoch * epochs * batch_size) / train_len)    # repeat count

In [ ]:
tf_data_tr = tf.data.Dataset.from_generator(data_generator_tr,
                                            output_signature=(tf.TensorSpec(shape=(36,), dtype=tf.float64),
                                                               tf.TensorSpec(shape=(36,), dtype=tf.float64))
                                            )

tf_data_tr = tf_data_tr.map(noise_adder,num_parallel_calls=tf.data.AUTOTUNE)\
                        .shuffle(buffer_size_tr)\
                        .batch(batch_size,num_parallel_calls=tf.data.AUTOTUNE)\
                        .repeat(repeat_count)\
                        .prefetch(tf.data.AUTOTUNE)

In [ ]:
tf_data_cv = tf.data.Dataset.from_generator(data_generator_cv,
                                            output_signature=(tf.TensorSpec(shape=(36,), dtype=tf.float64),
                                                               tf.TensorSpec(shape=(36,), dtype=tf.float64))
                                            )

# noise should not be added to the validation data or test data.
# https://machinelearningmastery.com/train-neural-networks-with-noise-to-reduce-overfitting/#:~:text=Noise%20is%20only%20added%20during%20training.%20No%20noise%20is%20added%20during%20the%20evaluation%20of%20the%20model
tf_data_cv = tf_data_cv.shuffle(buffer_size_cv)\
                        .batch(batch_size,num_parallel_calls=tf.data.AUTOTUNE)\
                        .prefetch(tf.data.AUTOTUNE)

In [ ]:
# https://www.tensorflow.org/tutorials/generative/autoencoder
latent_dim = 8

class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.encoder = tf.keras.Sequential([
      InputLayer(input_shape=(36,),name='l0'),                                 
      Dense(16,activation='sigmoid',name='l1'),                     
      Dense(latent_dim, activation='sigmoid',name='l2'),
    ],name='encoder')

    self.decoder = tf.keras.Sequential([
      Dense(16,activation='sigmoid',name='l3'),   
      Dense(36, activation='sigmoid',name='l4')
    ],name='decoder')

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder(latent_dim)

autoencoder.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

In [ ]:
autoencoder.fit(tf_data_tr,
                validation_data=tf_data_cv,
                epochs=epochs,
                steps_per_epoch=steps_per_epoch,
                validation_steps=validation_steps,
                verbose=1,
                use_multiprocessing=True,
                workers=4
                )

Epoch 1/5
185291/185291 [==============================] - 1949s 10ms/step - loss: 0.0063 - val_loss: 0.0031
Epoch 2/5
185291/185291 [==============================] - 1843s 10ms/step - loss: 0.0046 - val_loss: 0.0025
Epoch 3/5
185291/185291 [==============================] - 1837s 10ms/step - loss: 0.0045 - val_loss: 0.0024
Epoch 4/5
185291/185291 [==============================] - 1861s 10ms/step - loss: 0.0045 - val_loss: 0.0022
Epoch 5/5
185291/185291 [==============================] - 1854s 10ms/step - loss: 0.0045 - val_loss: 0.0023


In [ ]:
# Save the entire model as a SavedModel.
"""
!mkdir -p saved_model
autoencoder.save('saved_model/my_model')
"""

INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [ ]:
"""
!zip -r ./autoencoder.zip ./saved_model
"""

  adding: saved_model/ (stored 0%)
  adding: saved_model/my_model/ (stored 0%)
  adding: saved_model/my_model/keras_metadata.pb (deflated 89%)
  adding: saved_model/my_model/variables/ (stored 0%)
  adding: saved_model/my_model/variables/variables.data-00000-of-00001 (deflated 20%)
  adding: saved_model/my_model/variables/variables.index (deflated 65%)
  adding: saved_model/my_model/saved_model.pb (deflated 89%)
  adding: saved_model/my_model/assets/ (stored 0%)


In [ ]:
!unzip ./autoencoder.zip

Archive:  ./autoencoder.zip
   creating: saved_model/
   creating: saved_model/my_model/
  inflating: saved_model/my_model/keras_metadata.pb  
   creating: saved_model/my_model/variables/
  inflating: saved_model/my_model/variables/variables.data-00000-of-00001  
  inflating: saved_model/my_model/variables/variables.index  
  inflating: saved_model/my_model/saved_model.pb  
   creating: saved_model/my_model/assets/


In [ ]:
autoencoder = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (Sequential)         (None, 8)                 728       
_________________________________________________________________
decoder (Sequential)         (None, 36)                756       
Total params: 1,484
Trainable params: 1,484
Non-trainable params: 0
_________________________________________________________________


In [ ]:
autoencoder.encoder.summary()
autoencoder.decoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
l1 (Dense)                   (None, 16)                592       
_________________________________________________________________
l2 (Dense)                   (None, 8)                 136       
Total params: 728
Trainable params: 728
Non-trainable params: 0
_________________________________________________________________
Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
l3 (Dense)                   (None, 16)                144       
_________________________________________________________________
l4 (Dense)                   (None, 36)                612       
Total params: 756
Trainable params: 756
Non-trainable params: 0
_________________________________________________________________


In [ ]:
encoded_ftrs = autoencoder.encoder.predict(dataset[ftrs])

In [ ]:
enc_ftr_names = ['enc1','enc2','enc3','enc4','enc5','enc6','enc7','enc8']
dataset[enc_ftr_names] = encoded_ftrs

In [ ]:
dataset.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13307953 entries, 0 to 13307952
Data columns (total 49 columns):
 #   Column                        Dtype   
---  ------                        -----   
 0   user_id                       uint32  
 1   product_id                    uint16  
 2   order_id                      uint32  
 3   eval_set                      category
 4   reordered                     float16 
 5   #reorders_u                   float64 
 6   #purchases_u                  float64 
 7   #first_purchases_u            float64 
 8   p(reorder|user)               float64 
 9   mean_#reorders                float64 
 10  median_#reorders              float64 
 11  min_#reorders                 float64 
 12  max_#reorders                 float64 
 13  mean_#purchases               float64 
 14  median_#purchases             float64 
 15  min_#purchases                float64 
 16  max_#purchases                float64 
 17  mean_#first_purchases         float64 
 18  

In [ ]:
%%time
# Saving dataset locally
dataset.to_feather('dataset.feather')

CPU times: user 8.96 s, sys: 1.46 s, total: 10.4 s
Wall time: 4.09 s


In [ ]:
kaggle_file_uploader(files = ['/content/dataset.feather'],
                     folder='data',
                     id = 'shubhamscifi/instacart',
                     title = 'instacart',
                     msg='demo',
                     first_time=False,
                     del_after_upload=True)

Data package template written to: data/dataset-metadata.json
Starting upload for file dataset.feather
100% 0.99G/0.99G [00:10<00:00, 106MB/s]
Upload successful: dataset.feather (1009MB)
Starting upload for file dataset-prev.feather
100% 603M/603M [00:06<00:00, 96.1MB/s]
Upload successful: dataset-prev.feather (603MB)
Dataset version is being created. Please check progress at /api/v1/datasets/status/shubhamscifi/instacart


In [ ]:
metadata = dataset.columns[:4]
# 4th index is the target variable
continuous_ftrs = list(dataset.columns[5:27]) + list(dataset.columns[28:38]) + list(dataset.columns[41:49])
cat_ftrs = ['eatable',
            'reordered_in_last_order',
            'reordered_in_2ndlast_order',
            'reordered_in_3rdlast_order']
ftrs = continuous_ftrs + cat_ftrs

In [ ]:
print('No. of continous ftrs:'.ljust(24),len(continuous_ftrs))
print('No. of categorical ftrs:'.ljust(24),len(cat_ftrs))
print('Total no. of ftrs:'.ljust(24),len(ftrs))

No. of continous ftrs:   40
No. of categorical ftrs: 4
Total no. of ftrs:       44




---

